<a href="https://colab.research.google.com/github/DevDigit/git_test/blob/main/Copy_of_Dick_Pic_Editor_v0_1_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This Dick Pic Does Not Exist - Editor (Tensorflow Version)


### To run:
**Click "Open in Playground" above (if you see that option) and then Runtime > Run All and wait a few minutes for everything to load. (Takes about 4 minutes)**

Feel free to read the code (by clicking the triangle in the next section) while you wait.

---


### Credits

Model trained by [@DickExist](https://twitter.com/DickExist) from [TDPDNE](https://thisdickpicdoesnotexist.com).

Latent directions were discovered using [@harskish](https://twitter.com/harskish)'s [Ganspace](https://github.com/harskish/ganspace) [notebook](https://colab.research.google.com/github/harskish/ganspace/blob/master/notebooks/Ganspace_colab.ipynb) put together by [@realmeatyhuman](https://twitter.com/realmeatyhuman).

Editor notebook by [@arfafax](https://twitter.com/arfafax) \([Patreon](https://www.patreon.com/arfafax)).



---

Support the creator of This Dick Pic Does Not Exist on Ko-fi.
<div>
<a href="https://ko-fi.com/beezeetee">
<img src="https://cdn.ko-fi.com/cdn/kofi3.png?v=2" width="220"/>
</a>
</div>




# New section

## <- Click the triangle to view the code while you wait for it to load


In [18]:
#!git clone https://github.com/shawwn/stylegan2 -b estimator /content/stylegan2
!git clone https://github.com/cyrilzakka/stylegan2-tpu /content/stylegan2

fatal: destination path '/content/stylegan2' already exists and is not an empty directory.


In [19]:
import gdown
gdown.download('https://drive.google.com/uc?id=10bSRAB8_L8GUqLI4oNggTTBbKiirQUio', '/content/tdpdne-directions.zip', quiet=False) #https://drive.google.com/file/d/10bSRAB8_L8GUqLI4oNggTTBbKiirQUio/view?usp=sharing
gdown.download('https://drive.google.com/uc?id=13R0q8w3xg6CGPtygO85Bna_h0SVnH6Gy', '/content/network-tdpdne.pkl', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=10bSRAB8_L8GUqLI4oNggTTBbKiirQUio
To: /content/tdpdne-directions.zip
100%|██████████| 1.06M/1.06M [00:00<00:00, 67.1MB/s]

Access denied with the following error:




 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=13R0q8w3xg6CGPtygO85Bna_h0SVnH6Gy 



In [22]:
%tensorflow_version 2.x
%cd /content/stylegan2

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
/content/stylegan2


In [25]:
# Copyright (c) 2019, NVIDIA CORPORATION. All rights reserved.
#
# This work is licensed under the Creative Commons Attribution-NonCommercial
# 4.0 International License. To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

"""Miscellaneous utility classes and functions."""

import ctypes
import fnmatch
import importlib
import inspect
import numpy as np
import os
import shutil
import sys
import types
import io
import pickle
import re
import requests
import html
import hashlib
import glob
import uuid

from distutils.util import strtobool
from typing import Any, List, Tuple, Union


# Util classes
# ------------------------------------------------------------------------------------------


class EasyDict(dict):
    """Convenience class that behaves like a dict but allows access with the attribute syntax."""

    def __getattr__(self, name: str) -> Any:
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name: str, value: Any) -> None:
        self[name] = value

    def __delattr__(self, name: str) -> None:
        del self[name]


class Logger(object):
    """Redirect stderr to stdout, optionally print stdout to a file, and optionally force flushing on both stdout and the file."""

    def __init__(self, file_name: str = None, file_mode: str = "w", should_flush: bool = True):
        self.file = None

        if file_name is not None:
            self.file = open(file_name, file_mode)

        self.should_flush = should_flush
        self.stdout = sys.stdout
        self.stderr = sys.stderr

        sys.stdout = self
        sys.stderr = self

    def __enter__(self) -> "Logger":
        return self

    def __exit__(self, exc_type: Any, exc_value: Any, traceback: Any) -> None:
        self.close()

    def write(self, text: str) -> None:
        """Write text to stdout (and a file) and optionally flush."""
        if len(text) == 0: # workaround for a bug in VSCode debugger: sys.stdout.write(''); sys.stdout.flush() => crash
            return

        if self.file is not None:
            self.file.write(text)

        self.stdout.write(text)

        if self.should_flush:
            self.flush()

    def flush(self) -> None:
        """Flush written text to both stdout and a file, if open."""
        if self.file is not None:
            self.file.flush()

        self.stdout.flush()

    def close(self) -> None:
        """Flush, close possible files, and remove stdout/stderr mirroring."""
        self.flush()

        # if using multiple loggers, prevent closing in wrong order
        if sys.stdout is self:
            sys.stdout = self.stdout
        if sys.stderr is self:
            sys.stderr = self.stderr

        if self.file is not None:
            self.file.close()


# Small util functions
# ------------------------------------------------------------------------------------------


def format_time(seconds: Union[int, float]) -> str:
    """Convert the seconds to human readable string with days, hours, minutes and seconds."""
    s = int(np.rint(seconds))

    if s < 60:
        return "{0}s".format(s)
    elif s < 60 * 60:
        return "{0}m {1:02}s".format(s // 60, s % 60)
    elif s < 24 * 60 * 60:
        return "{0}h {1:02}m {2:02}s".format(s // (60 * 60), (s // 60) % 60, s % 60)
    else:
        return "{0}d {1:02}h {2:02}m".format(s // (24 * 60 * 60), (s // (60 * 60)) % 24, (s // 60) % 60)


def ask_yes_no(question: str) -> bool:
    """Ask the user the question until the user inputs a valid answer."""
    while True:
        try:
            print("{0} [y/n]".format(question))
            return strtobool(input().lower())
        except ValueError:
            pass


def tuple_product(t: Tuple) -> Any:
    """Calculate the product of the tuple elements."""
    result = 1

    for v in t:
        result *= v

    return result


_str_to_ctype = {
    "uint8": ctypes.c_ubyte,
    "uint16": ctypes.c_uint16,
    "uint32": ctypes.c_uint32,
    "uint64": ctypes.c_uint64,
    "int8": ctypes.c_byte,
    "int16": ctypes.c_int16,
    "int32": ctypes.c_int32,
    "int64": ctypes.c_int64,
    "float32": ctypes.c_float,
    "float64": ctypes.c_double
}


def get_dtype_and_ctype(type_obj: Any) -> Tuple[np.dtype, Any]:
    """Given a type name string (or an object having a __name__ attribute), return matching Numpy and ctypes types that have the same size in bytes."""
    type_str = None

    if isinstance(type_obj, str):
        type_str = type_obj
    elif hasattr(type_obj, "__name__"):
        type_str = type_obj.__name__
    elif hasattr(type_obj, "name"):
        type_str = type_obj.name
    else:
        raise RuntimeError("Cannot infer type name from input")

    assert type_str in _str_to_ctype.keys()

    my_dtype = np.dtype(type_str)
    my_ctype = _str_to_ctype[type_str]

    assert my_dtype.itemsize == ctypes.sizeof(my_ctype)

    return my_dtype, my_ctype


def is_pickleable(obj: Any) -> bool:
    try:
        with io.BytesIO() as stream:
            pickle.dump(obj, stream)
        return True
    except:
        return False


# Functionality to import modules/objects by name, and call functions by name
# ------------------------------------------------------------------------------------------

def get_module_from_obj_name(obj_name: str) -> Tuple[types.ModuleType, str]:
    """Searches for the underlying module behind the name to some python object.
    Returns the module and the object name (original name with module part removed)."""

    # allow convenience shorthands, substitute them by full names
    obj_name = re.sub("^np.", "numpy.", obj_name)
    obj_name = re.sub("^tf.", "tensorflow.", obj_name)

    # list alternatives for (module_name, local_obj_name)
    parts = obj_name.split(".")
    name_pairs = [(".".join(parts[:i]), ".".join(parts[i:])) for i in range(len(parts), 0, -1)]

    # try each alternative in turn
    for module_name, local_obj_name in name_pairs:
        try:
            module = importlib.import_module(module_name) # may raise ImportError
            get_obj_from_module(module, local_obj_name) # may raise AttributeError
            return module, local_obj_name
        except:
            pass

    # maybe some of the modules themselves contain errors?
    for module_name, _local_obj_name in name_pairs:
        try:
            importlib.import_module(module_name) # may raise ImportError
        except ImportError:
            if not str(sys.exc_info()[1]).startswith("No module named '" + module_name + "'"):
                raise

    # maybe the requested attribute is missing?
    for module_name, local_obj_name in name_pairs:
        try:
            module = importlib.import_module(module_name) # may raise ImportError
            get_obj_from_module(module, local_obj_name) # may raise AttributeError
        except ImportError:
            pass

    # we are out of luck, but we have no idea why
    raise ImportError(obj_name)


def get_obj_from_module(module: types.ModuleType, obj_name: str) -> Any:
    """Traverses the object name and returns the last (rightmost) python object."""
    if obj_name == '':
        return module
    obj = module
    for part in obj_name.split("."):
        obj = getattr(obj, part)
    return obj


def get_obj_by_name(name: str) -> Any:
    """Finds the python object with the given name."""
    module, obj_name = get_module_from_obj_name(name)
    return get_obj_from_module(module, obj_name)


def call_func_by_name(*args, func_name: str = None, **kwargs) -> Any:
    """Finds the python object with the given name and calls it as a function."""
    assert func_name is not None
    func_obj = get_obj_by_name(func_name)
    assert callable(func_obj)
    return func_obj(*args, **kwargs)


def get_module_dir_by_obj_name(obj_name: str) -> str:
    """Get the directory path of the module containing the given object name."""
    module, _ = get_module_from_obj_name(obj_name)
    return os.path.dirname(inspect.getfile(module))


def is_top_level_function(obj: Any) -> bool:
    """Determine whether the given object is a top-level function, i.e., defined at module scope using 'def'."""
    return callable(obj) and obj.__name__ in sys.modules[obj.__module__].__dict__


def get_top_level_function_name(obj: Any) -> str:
    """Return the fully-qualified name of a top-level function."""
    assert is_top_level_function(obj)
    return obj.__module__ + "." + obj.__name__


# File system helpers
# ------------------------------------------------------------------------------------------

def list_dir_recursively_with_ignore(dir_path: str, ignores: List[str] = None, add_base_to_relative: bool = False) -> List[Tuple[str, str]]:
    """List all files recursively in a given directory while ignoring given file and directory names.
    Returns list of tuples containing both absolute and relative paths."""
    assert os.path.isdir(dir_path)
    base_name = os.path.basename(os.path.normpath(dir_path))

    if ignores is None:
        ignores = []

    result = []

    for root, dirs, files in os.walk(dir_path, topdown=True):
        for ignore_ in ignores:
            dirs_to_remove = [d for d in dirs if fnmatch.fnmatch(d, ignore_)]

            # dirs need to be edited in-place
            for d in dirs_to_remove:
                dirs.remove(d)

            files = [f for f in files if not fnmatch.fnmatch(f, ignore_)]

        absolute_paths = [os.path.join(root, f) for f in files]
        relative_paths = [os.path.relpath(p, dir_path) for p in absolute_paths]

        if add_base_to_relative:
            relative_paths = [os.path.join(base_name, p) for p in relative_paths]

        assert len(absolute_paths) == len(relative_paths)
        result += zip(absolute_paths, relative_paths)

    return result


def copy_files_and_create_dirs(files: List[Tuple[str, str]]) -> None:
    """Takes in a list of tuples of (src, dst) paths and copies files.
    Will create all necessary directories."""
    for file in files:
        target_dir_name = os.path.dirname(file[1])

        # will create all intermediate-level directories
        if not os.path.exists(target_dir_name):
            os.makedirs(target_dir_name)

        shutil.copyfile(file[0], file[1])


# URL helpers
# ------------------------------------------------------------------------------------------

def is_url(obj: Any) -> bool:
    """Determine whether the given object is a valid URL string."""
    if not isinstance(obj, str) or not "://" in obj:
        return False
    try:
        res = requests.compat.urlparse(obj)
        if not res.scheme or not res.netloc or not "." in res.netloc:
            return False
        res = requests.compat.urlparse(requests.compat.urljoin(obj, "/"))
        if not res.scheme or not res.netloc or not "." in res.netloc:
            return False
    except:
        return False
    return True


def open_url(url: str, cache_dir: str = None, num_attempts: int = 10, verbose: bool = True) -> Any:
    """Download the given URL and return a binary-mode file object to access the data."""
    assert is_url(url)
    assert num_attempts >= 1

    # Lookup from cache.
    url_md5 = hashlib.md5(url.encode("utf-8")).hexdigest()
    if cache_dir is not None:
        cache_files = glob.glob(os.path.join(cache_dir, url_md5 + "_*"))
        if len(cache_files) == 1:
            return open(cache_files[0], "rb")

    # Download.
    url_name = None
    url_data = None
    with requests.Session() as session:
        if verbose:
            print("Downloading %s ..." % url, end="", flush=True)
        for attempts_left in reversed(range(num_attempts)):
            try:
                with session.get(url) as res:
                    res.raise_for_status()
                    if len(res.content) == 0:
                        raise IOError("No data received")

                    if len(res.content) < 8192:
                        content_str = res.content.decode("utf-8")
                        if "download_warning" in res.headers.get("Set-Cookie", ""):
                            links = [html.unescape(link) for link in content_str.split('"') if "export=download" in link]
                            if len(links) == 1:
                                url = requests.compat.urljoin(url, links[0])
                                raise IOError("Google Drive virus checker nag")
                        if "Google Drive - Quota exceeded" in content_str:
                            raise IOError("Google Drive quota exceeded")

                    match = re.search(r'filename="([^"]*)"', res.headers.get("Content-Disposition", ""))
                    url_name = match[1] if match else url
                    url_data = res.content
                    if verbose:
                        print(" done")
                    break
            except:
                if not attempts_left:
                    if verbose:
                        print(" failed")
                    raise
                if verbose:
                    print(".", end="", flush=True)

    # Save to cache.
    if cache_dir is not None:
        safe_name = re.sub(r"[^0-9a-zA-Z-._]", "_", url_name)
        cache_file = os.path.join(cache_dir, url_md5 + "_" + safe_name)
        temp_file = os.path.join(cache_dir, "tmp_" + uuid.uuid4().hex + "_" + url_md5 + "_" + safe_name)
        os.makedirs(cache_dir, exist_ok=True)
        with open(temp_file, "wb") as f:
            f.write(url_data)
        os.replace(temp_file, cache_file) # atomic

    # Return data as file object.
    return io.BytesIO(url_data)

In [26]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy
import tensorflow as tf
import tflex

ModuleNotFoundError: ignored

In [27]:
if 'COLAB_TPU_ADDR' in os.environ:
    os.environ['TPU_NAME'] = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    os.environ['NOISY'] = '1'

In [28]:
tflib.init_tf()
sess = tf.get_default_session()
sess.list_devices()
cores = tflex.get_cores()
tflex.set_override_cores(cores)
_G, _D, Gs = pickle.load(open("/content/network-tdpdne.pkl", "rb"))
# _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
# _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
# Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

NameError: ignored

In [29]:
 def generate_image_from_w(w, truncation_psi):
    with tflex.device('/gpu:0'):
        noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
        Gs_kwargs = dnnlib.EasyDict()
        Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
        Gs_kwargs.randomize_noise = False
        if truncation_psi is not None:
            Gs_kwargs.truncation_psi = truncation_psi
        synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation_psi, minibatch_size=1)
        images = Gs.components.synthesis.run(w, randomize_noise=False, **synthesis_kwargs)
        display(PIL.Image.fromarray(images[0], 'RGB').resize((500,500),Image.LANCZOS))

In [30]:
%cd "/content"

/content


In [31]:
%mkdir directions
%cd directions
!unzip /content/tdpdne-directions.zip

/content/directions
Archive:  /content/tdpdne-directions.zip
  inflating: c30.4_layers_12_15.npy  
  inflating: c31.2_layers_6_9.npy    
  inflating: c79.2_layers_6_9.npy    
  inflating: c72.2_layers_6_9.npy    
  inflating: c67.5_layers_15_18.npy  
  inflating: c77.2_layers_6_9.npy    
  inflating: c14.5_layers_15_18.npy  
  inflating: c42.1_layers_3_6.npy    
  inflating: c08.3_layers_9_12.npy   
  inflating: c57.4_layers_12_15.npy  
  inflating: c01.4_layers_12_15.npy  
  inflating: c10.3_layers_9_12.npy   
  inflating: c08.0_layers_0_3.npy    
  inflating: c27.3_layers_9_12.npy   
  inflating: c33.3_layers_9_12.npy   
  inflating: c22.3_layers_9_12.npy   
  inflating: c27.2_layers_6_9.npy    
  inflating: c39.2_layers_6_9.npy    
  inflating: c07.0_layers_0_3.npy    
  inflating: c31.5_layers_15_18.npy  
  inflating: c64.2_layers_6_9.npy    
  inflating: c69.4_layers_12_15.npy  
  inflating: c15.2_layers_6_9.npy    
  inflating: c56.3_layers_9_12.npy   
  inflating: c16.4_layers_1

In [32]:
!rm /content/directions/StyleGAN2-Light_direction-ffhq-ipca-w-style-comp15-range8-9.pkl
!rm -r /content/directions/.ipynb_checkpoints
#!mv /content/directions/directions/* /content/ganspace/directions
#!rm -r /content/directions/directions

rm: cannot remove '/content/directions/.ipynb_checkpoints': No such file or directory


In [33]:
import os
named_directions = {}
latent_dirs = []
starts = []
ends = []
# 
path_to_directions = "/content/directions"

# This loads the directions in a dictionary in this format:
# {'name' : [direction_num, start, end]}
# and you load the direction by:
# directions[direction_num]

for i,file in enumerate(sorted(os.listdir(path_to_directions))):
    np_file = np.load(f'{path_to_directions}/{file}', allow_pickle=True)
    name = file.split("_layers_")[0].split("/")[-1]
    file = file.split('_')
    comp = int(name.split(".")[0][1:])
    
    named_directions[f'{name}'] = [comp, int(file[-2]), int(file[-1].split('.')[0]), f'{name}']
    if int(file[-2]) == 0:
        latent_dirs.append(np_file)


In [34]:
named_directions

{'c00.0': [0, 0, 3, 'c00.0'],
 'c00.1': [0, 3, 6, 'c00.1'],
 'c00.2': [0, 6, 9, 'c00.2'],
 'c00.3': [0, 9, 12, 'c00.3'],
 'c00.4': [0, 12, 15, 'c00.4'],
 'c00.5': [0, 15, 18, 'c00.5'],
 'c01.0': [1, 0, 3, 'c01.0'],
 'c01.1': [1, 3, 6, 'c01.1'],
 'c01.2': [1, 6, 9, 'c01.2'],
 'c01.3': [1, 9, 12, 'c01.3'],
 'c01.4': [1, 12, 15, 'c01.4'],
 'c01.5': [1, 15, 18, 'c01.5'],
 'c02.0': [2, 0, 3, 'c02.0'],
 'c02.1': [2, 3, 6, 'c02.1'],
 'c02.2': [2, 6, 9, 'c02.2'],
 'c02.3': [2, 9, 12, 'c02.3'],
 'c02.4': [2, 12, 15, 'c02.4'],
 'c02.5': [2, 15, 18, 'c02.5'],
 'c03.0': [3, 0, 3, 'c03.0'],
 'c03.1': [3, 3, 6, 'c03.1'],
 'c03.2': [3, 6, 9, 'c03.2'],
 'c03.3': [3, 9, 12, 'c03.3'],
 'c03.4': [3, 12, 15, 'c03.4'],
 'c03.5': [3, 15, 18, 'c03.5'],
 'c04.0': [4, 0, 3, 'c04.0'],
 'c04.1': [4, 3, 6, 'c04.1'],
 'c04.2': [4, 6, 9, 'c04.2'],
 'c04.3': [4, 9, 12, 'c04.3'],
 'c04.4': [4, 12, 15, 'c04.4'],
 'c04.5': [4, 15, 18, 'c04.5'],
 'c05.0': [5, 0, 3, 'c05.0'],
 'c05.1': [5, 3, 6, 'c05.1'],
 'c05.2': [5, 6

In [35]:
import yaml
%mkdir /content/animations
%cd /content/animations

/content/animations


# UI

Paste a slider config below and run this cell to load that config:

In [36]:
load_directions = """
c00.0: [0, 0, 3, c00.0]
c00.1: [0, 3, 6, c00.1]
c00.2: [0, 6, 9, c00.2]
c00.3: [0, 9, 12, c00.3]
c00.4: [0, 12, 15, c00.4]
c00.5: [0, 15, 16, c00.5]
c01.0: [1, 0, 3, c01.0]
c01.1: [1, 3, 6, c01.1]
c01.2: [1, 6, 9, c01.2]
c01.3: [1, 9, 12, c01.3]
c01.4: [1, 12, 15, c01.4]
c01.5: [1, 15, 16, c01.5]
c02.0: [2, 0, 3, c02.0]
c02.1: [2, 3, 6, c02.1]
c02.2: [2, 6, 9, c02.2]
c02.3: [2, 9, 12, c02.3]
c02.4: [2, 12, 15, c02.4]
c02.5: [2, 15, 16, c02.5]
c03.0: [3, 0, 3, c03.0]
c03.1: [3, 3, 6, c03.1]
c03.2: [3, 6, 9, c03.2]
c03.3: [3, 9, 12, c03.3]
c03.4: [3, 12, 15, c03.4]
c03.5: [3, 15, 16, c03.5]
c04.0: [4, 0, 3, c04.0]
c04.1: [4, 3, 6, c04.1]
c04.2: [4, 6, 9, c04.2]
c04.3: [4, 9, 12, c04.3]
c04.4: [4, 12, 15, c04.4]
c04.5: [4, 15, 16, c04.5]
c05.0: [5, 0, 3, c05.0]
c05.1: [5, 3, 6, c05.1]
c05.2: [5, 6, 9, c05.2]
c05.3: [5, 9, 12, c05.3]
c05.4: [5, 12, 15, c05.4]
c05.5: [5, 15, 16, c05.5]
c06.0: [6, 0, 3, c06.0]
c06.1: [6, 3, 6, c06.1]
c06.2: [6, 6, 9, c06.2]
c06.3: [6, 9, 12, c06.3]
c06.4: [6, 12, 15, c06.4]
c06.5: [6, 15, 16, c06.5]
c07.0: [7, 0, 3, c07.0]
c07.1: [7, 3, 6, c07.1]
c07.2: [7, 6, 9, c07.2]
c07.3: [7, 9, 12, c07.3]
c07.4: [7, 12, 15, c07.4]
c07.5: [7, 15, 16, c07.5]
c08.0: [8, 0, 3, c08.0]
c08.1: [8, 3, 6, c08.1]
c08.2: [8, 6, 9, c08.2]
c08.3: [8, 9, 12, c08.3]
c08.4: [8, 12, 15, c08.4]
c08.5: [8, 15, 16, c08.5]
c09.0: [9, 0, 3, c09.0]
c09.1: [9, 3, 6, c09.1]
c09.2: [9, 6, 9, c09.2]
c09.3: [9, 9, 12, c09.3]
c09.4: [9, 12, 15, c09.4]
c09.5: [9, 15, 16, c09.5]
c10.0: [10, 0, 3, c10.0]
c10.1: [10, 3, 6, c10.1]
c10.2: [10, 6, 9, c10.2]
c10.3: [10, 9, 12, c10.3]
c10.4: [10, 12, 15, c10.4]
c10.5: [10, 15, 16, c10.5]
c11.0: [11, 0, 3, c11.0]
c11.1: [11, 3, 6, c11.1]
c11.2: [11, 6, 9, c11.2]
c11.3: [11, 9, 12, c11.3]
c11.4: [11, 12, 15, c11.4]
c11.5: [11, 15, 16, c11.5]
c12.0: [12, 0, 3, c12.0]
c12.1: [12, 3, 6, c12.1]
c12.2: [12, 6, 9, c12.2]
c12.3: [12, 9, 12, c12.3]
c12.4: [12, 12, 15, c12.4]
c12.5: [12, 15, 16, c12.5]
c13.0: [13, 0, 3, c13.0]
c13.1: [13, 3, 6, c13.1]
c13.2: [13, 6, 9, c13.2]
c13.3: [13, 9, 12, c13.3]
c13.4: [13, 12, 15, c13.4]
c13.5: [13, 15, 16, c13.5]
c14.0: [14, 0, 3, c14.0]
c14.1: [14, 3, 6, c14.1]
c14.2: [14, 6, 9, c14.2]
c14.3: [14, 9, 12, c14.3]
c14.4: [14, 12, 15, c14.4]
c14.5: [14, 15, 16, c14.5]
c15.0: [15, 0, 3, c15.0]
c15.1: [15, 3, 6, c15.1]
c15.2: [15, 6, 9, c15.2]
c15.3: [15, 9, 12, c15.3]
c15.4: [15, 12, 15, c15.4]
c15.5: [15, 15, 16, c15.5]
"""
named_directions = yaml.load(load_directions)


TypeError: ignored

In [48]:
#@title Run UI (make sure you've done Runtime > Run All first or it won't work)
from ipywidgets import fixed
import PIL
import numpy as np
import ipywidgets as widgets
from PIL import Image
from IPython.display import clear_output
from ipywidgets import Layout, Button, Box, VBox, Label

box_layout = Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='',
                    height="500px")

loaded_w = None
all_w = None
def display_sample(seed, truncation, direction, distance, scale, start, end, update, disp=True, save=None, noise_spec=None, **args):
    global loaded_w
    global all_w
    if update == False:
        print("False")
    # blockPrint()
    rng = np.random.RandomState(seed)
    z = rng.standard_normal(*Gs.input_shape[1:]).reshape(1, *Gs.input_shape[1:])
    #z = np.load("/content/drive/My Drive/latents/1006-0-0-9.npy")
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.randomize_noise = False

    all_w = Gs.components.mapping.run(z, None, **Gs_kwargs) # [minibatch, layer, component]
    if loaded_w is not None:
      all_w = loaded_w.copy()
    
    for i, item in enumerate(args):
        value = args[item]
        start_l = named_directions[item][1]
        end_l = named_directions[item][2]
        direction_l = latent_dirs[named_directions[item][0]]
        for l in range(start_l, end_l):
            all_w[0][l] = all_w[0][l] + direction_l * value * scale

    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]

    generate_image_from_w(all_w, truncation)

seed = np.random.randint(0,100000)
style = {'description_width': 'initial', 'width': '150px'}
row_length = 6

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.4, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)
start_layer = widgets.IntSlider(min=0, max=18, step=1, value=18, description='start layer: ', continuous_update=False)
end_layer = widgets.IntSlider(min=0, max=18, step=1, value=18, description='end layer: ', continuous_update=False)


update = widgets.Checkbox(value=True, description="update")

directions_list = []
params = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}

for i, item in enumerate(named_directions):
    name = named_directions[item][3]
    widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=0, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
    directions_list.append(widget)
    params[item] = widget

top_box = widgets.HBox([seed, truncation])
rename_dropdown = widgets.Dropdown(options=named_directions.keys())
rename_box = widgets.Text()
rename = widgets.Button(description="Rename")
bot_box = widgets.HBox([rename_dropdown, rename_box, rename])

ui = widgets.VBox([top_box, bot_box])

grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
for i in range(len(directions_list)//6):
    for j in range(6):
        grid[i, j] = directions_list[6*i+j]
    
ui2 = grid


random = widgets.Button(description="Randomize Sliders")
reset = widgets.Button(description="Reset Sliders")
mobile = widgets.Button(description="Mobile Mode")
desktop = widgets.Button(description="Desktop Mode")
def reset_sliders(b):
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=0, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
    for i in range(len(directions_list)//6):
        for j in range(6):
            grid[i, j] = directions_list[6*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    last_button = mobile
    if row_length == 1:
        last_button = desktop
    display(ui, out, ui2, reset, random, last_button, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

def random_sliders(b):
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=np.random.normal(scale=2.5), description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
    for i in range(len(directions_list)//6):
        for j in range(6):
            grid[i, j] = directions_list[6*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    last_button = mobile
    if row_length == 1:
        last_button = desktop
    display(ui, out, ui2, reset, random, last_button, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

def mobile_mode(b):
    global row_length
    global params
    row_length = 1
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=params[item].value, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    row_list = []
    foo = []
    for i, item in enumerate(directions_list):
        row_list.append(item)
        if len(row_list) == row_length:
            bar = widgets.HBox(row_list)
            foo.append(bar)
            row_list = []
    bar = widgets.HBox(row_list)
    foo.append(bar)

    ui2 = widgets.VBox(foo)
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    display(ui, out, ui2, reset, random, desktop, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

def desktop_mode(b):
    global row_length
    global params
    row_length = 6
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=params[item].value, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
    for i in range(len(directions_list)//6):
        for j in range(6):
            grid[i, j] = directions_list[6*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    display(ui, out, ui2, reset, random, mobile, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

def rename_slider(b):
    global row_length
    global params
    print(rename_dropdown.value, rename_box.value)
    named_directions[rename_dropdown.value][3] = rename_box.value
    #del named_directions[rename_dropdown.value]
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        value_new = 0
        if item in params:
            value_new = params[item].value
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=value_new, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//6, 6, width='90%', height="300px")
    for i in range(len(directions_list)//6):
        for j in range(6):
            grid[i, j] = directions_list[6*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    display(ui, out, ui2, reset, random, mobile, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))
  
save_name = widgets.Text()
load_name = widgets.Text()
save_btn = widgets.Button(description="Save")
load_btn = widgets.Button(description="Load")

def load_w(b):
    global loaded_w
    print("Loading ", load_name.value + ".npy")
    loaded_w = np.load(load_name.value + ".npy")

def save_w(b):
    global loaded_w
    global all_w
    if save_name.value:
        out_name = save_name.value
    np.save(out_name + "orig.npy", all_w)
    print("saved ", out_name + "orig.npy")
    generate_image_from_w(all_w, 1.0)
    global params
    rng = np.random.RandomState(params['seed'].value)
    z = rng.standard_normal(*Gs.input_shape[1:]).reshape(1, *Gs.input_shape[1:])

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.randomize_noise = False

    all_w = Gs.components.mapping.run(z, None, **Gs_kwargs) # [minibatch, layer, component]
    if loaded_w is not None:
        print("Loaded w")
        all_w = loaded_w.copy()

    for i, item in enumerate(params):
        if item not in ['seed', 'truncation', 'direction', 'distance', 'scale', 'start', 'end', 'update']:
            value = params[item].value
            start_l = named_directions[item][1]
            end_l = named_directions[item][2]
            direction_l = latent_dirs[named_directions[item][0]]
            for l in range(start_l, end_l):
                all_w[0][l] = all_w[0][l] + direction_l * value * params['scale'].value

    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * params['truncation'].value # [minibatch, layer, component]

    from datetime import datetime
    now = datetime.now()

    out_name = now.isoformat()
    
    if save_name.value:
        out_name = save_name.value
    np.save(out_name + ".npy", all_w)
    print("saved ", out_name + ".npy")
    generate_image_from_w(all_w, 1.0)

random.on_click(random_sliders)
reset.on_click(reset_sliders)
mobile.on_click(mobile_mode)
desktop.on_click(desktop_mode)
rename.on_click(rename_slider)
save_btn.on_click(save_w)
load_btn.on_click(load_w)
out = widgets.interactive_output(display_sample, params)
display(ui, out, ui2, reset, random, mobile, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))



Output()

GridspecLayout(children=(FloatSlider(value=0.0, continuous_update=False, description='c00.0: ', layout=Layout(…

Button(description='Reset Sliders', style=ButtonStyle())

Button(description='Randomize Sliders', style=ButtonStyle())

Button(description='Mobile Mode', style=ButtonStyle())

In [ ]:
import yaml
print(yaml.dump(named_directions))

# Animation stuff

In [41]:
import math
from PIL import ImageFont
from PIL import ImageDraw
def interpolate_between_ws(seed_array, truncation=0.5, duration_sec = 12.0, smoothing_sec = 1.0, mp4_fps = 20, filename=None, text=False):
    #_G, _D, Gs = pickle.load(open("/content/network-e621.pkl", "rb"))
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation, minibatch_size=8)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    #if seed_array[0] != seed_array[-1]:
    #    seed_array.append(seed_array[0])
    

    all_w = seed_array
        
    num_frames = int(np.rint(duration_sec * mp4_fps))
        
    def make_frame(t):
        blend = ((len(seed_array)-1)*t/duration_sec)%1.0
        src_i = math.floor((t/duration_sec)*(len(seed_array)-1))
        dst_i = src_i + 1
        #print(t, blend, src_i, dst_i)
        all_w_new = (blend * all_w[dst_i]) + (1 - blend) * all_w[src_i]
        all_images_src = Gs.components.synthesis.run(all_w_new, randomize_noise=False, **synthesis_kwargs)
        #all_images_dst = Gs.components.synthesis.run(all_w_dst, randomize_noise=False, **synthesis_kwargs)
        if text:
            new_im = PIL.Image.new('RGB', (512, 600))
            new_im.paste(PIL.Image.fromarray(np.median(all_images_src, axis=0).astype(np.uint8)), (0, 0))
            draw = ImageDraw.Draw(new_im)
            font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", size=16)
            draw.text((10, 512), "{:0.2f}".format((1-blend)), (255, 0, 0), font=font)
            draw.text((50, 512), str(seed_array[src_i]), (255, 255, 255), font=font)
            draw.text((10, 550), "{:0.2f}".format((blend)), (0, 255, 0), font=font)
            draw.text((50, 550), str(seed_array[dst_i]), (255, 255, 255), font=font)
            return np.array(new_im)
        else:
            return all_images_src[0]

    
    import moviepy.editor
    from datetime import datetime
    now = datetime.now()

    np.save(now.isoformat() + ".npy", all_w)
    mp4_file = '%s.mp4' % (now)
    if filename:
        mp4_file = filename
    mp4_codec = 'libx264'
    mp4_bitrate = '5M'

    video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
    video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)
    
    return mp4_file

In [ ]:
import glob
def preview_files(glob_str="s*.npy"):
    for file in sorted(glob.glob(glob_str)):
      try:
        truncation = 1
        print(file)
        w = np.load(file)
        noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
        Gs_kwargs = dnnlib.EasyDict()
        Gs_kwargs.randomize_noise = False


        if truncation != 1:
            w_avg = Gs.get_var('dlatent_avg')
            w = w_avg + (w - w_avg) * truncation # [minibatch, layer, component]

        generate_image_from_w(w, truncation)
      except:
        print("Error")

# Generate animation

# New section

In [ ]:
!error Causing an error so the following cells are not run automatically....

Run this cell to preview all of the snapshots you've saved:

In [ ]:
preview_files("a*.npy")

List all of the files you want to animate between:

In [43]:
ws = '''
z22513.npy
z22513_2.npy
z22513.npy
z58160.npy
z33349.npy
z58160.npy
h66935.npy
z85844_2.npy
h66935.npy
z22513.npy
'''.split()
ws


['z22513.npy',
 'z22513_2.npy',
 'z22513.npy',
 'z58160.npy',
 'z33349.npy',
 'z58160.npy',
 'h66935.npy',
 'z85844_2.npy',
 'h66935.npy',
 'z22513.npy']

Run this to generate the animation:

In [44]:
w_arr = []
for w in ws:
    w_val = np.load(w)
    w_arr.append(w_val)

output_file = interpolate_between_ws(w_arr, duration_sec = 8.0, smoothing_sec = 1.0, mp4_fps = 30)

FileNotFoundError: ignored

Run this to download the animation:

In [45]:
from google.colab import files
files.download(output_file) 

NameError: ignored